In [ ]:
# https://github.com/Ko4eBHuK/nlp-23-spring/tree/main/tasks/task-03
!python --version

Python 3.9.16


In [ ]:
import regex as re
import os
import nltk
from nltk.corpus import stopwords
from google.colab import drive
import pandas as pd


drive.mount('/content/gdrive')

nltk.download('stopwords')

assets_url = 'gdrive/My Drive/Colab Notebooks/nlp-2023/assets/train/'
stops = set(stopwords.words('english'))

Mounted at /content/gdrive


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
token_freq_dict = dict()
file_names_contents_dict = dict()
S = 0

for address, dirs, files in os.walk(assets_url):
  for file_name in files:
    file_content = []
    with open(os.path.join(address, file_name), mode='r') as annotated_document_file:
      for sentence in annotated_document_file.read().split('\n\n'):
        for annotation in sentence.split('\n'):
          word_stem_lem = annotation.split('\t')
          if len(word_stem_lem) == 3:
            token = word_stem_lem[0]
            # Очистить полученные данные от знаков пунктуации. Можно использовать регулярное выражение: [^\P{P}-]+;
            if not re.match('[^\P{P}-]+', token):
              token = re.sub('[^\P{P}-]+', '', token) # Привести полученные данные к нижнему регистру;
              # Очистить полученные данные от стоп слов. Можно использовать nltk.corpus.stopwords;
              if not token in stops:
                # Словарь токенов с их частотами по всем данным
                token_freq_dict[token] = token_freq_dict.get(token, 0) + 1
                file_content.append(token)
                S += 1
    file_names_contents_dict[file_name] = file_content

In [ ]:
# read token_freq from file
assets_generated_url = 'gdrive/My Drive/Colab Notebooks/nlp-2023/assets/task-3/'

tokens_freq_df = pd.read_csv(assets_generated_url + 'token-freq.csv')

In [ ]:
tokens_freq_df

In [ ]:
# 1. Результаты необходимо сохранить во внешние файлы в произвольном формате
  # Сохранение словаря частот токенов
import csv

token_freq_csv_file_url = assets_generated_url + 'token-freq.csv'

with open(token_freq_csv_file_url, 'w') as csvfile:
        csvfile.write('token,frequency\n')
        for key, value in token_freq_dict.items():
            csvfile.write(f"{key},{value}\n")

In [ ]:
top_2k_tokens_by_freq = dict(sorted(token_freq_dict.items(), key=lambda x: x[1], reverse=True)[:2000])

In [ ]:
first_50k_docs_with_content = dict(list(file_names_contents_dict.items())[:50000])

In [ ]:
term_doc_matrix = pd.DataFrame(data=0, index=first_50k_docs_with_content.keys(), columns=top_2k_tokens_by_freq.keys())

for doc_name, tokens in first_50k_docs_with_content.items():
  all_tokens = len(tokens)
  for token in tokens:
    if token in top_2k_tokens_by_freq.keys():
      term_doc_matrix[token][doc_name] += 1

In [ ]:
# 1. Результаты необходимо сохранить во внешние файлы в произвольном формате
  # Сохранение term-doc-matrix
term_doc_matrix.to_csv(assets_generated_url + 'term-doc-matrix.csv')

In [ ]:
# read token_freq dictionary and term_doc_matrix from files

In [ ]:
# 2. Реализовать один из базовых методов векторизации произвольного текста
# Разработать метод, позволяющий преобразовать произвольный текст в вектор значений tf-idf,
  # с использованием словаря наиболее частых слов и матрицы "термин-документ", полученных ранее (на шаге 1);

import math

def get_TF_vec_of_doc(doc_text, available_tokens):
  doc_tokens = re.findall(r'[^\s.!?\-;:]+', doc_text)
  TF_vec = pd.DataFrame(data=0, index=[0], columns=available_tokens)
  for doc_token in doc_tokens:
    if doc_token in available_tokens:
      TF_vec[doc_token] += 1
  for token in available_tokens:
    TF_vec[token] = TF_vec[token]/len(available_tokens)
  return TF_vec

def getIDF_vec(term_doc_matrix, available_tokens):
  IDF_vec = pd.DataFrame(data=0, index=[0], columns=available_tokens.keys())
  for token in available_tokens:
    for index, row in term_doc_matrix.iterrows():
      if row[token] != 0:
        IDF_vec[token] += 1
  for token in available_tokens:
    IDF_vec[token] = math.log(term_doc_matrix.shape[0] / IDF_vec[token])
  return IDF_vec

In [ ]:
get_TF_vec_of_doc('Bebkiovan shmebra wants new anime. Such  two two tow W W W W W W W W W W W W W W W W Wunicly, only it! Reks shemks gres.', top_2k_tokens_by_freq.keys())

,-,The,US,said,new,A,year,first,company,two,...,century,traffic,W,seeded,semi,rain,vendors,hour,stadium,death
0,0.0,0.0,0.0,0.0,0.0005,0.0,0.0,0.0,0.0,0.001,...,0.0,0.0,0.008,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
getIDF_vec(term_doc_matrix.head(15), top_2k_tokens_by_freq)

,-,The,US,said,new,A,year,first,company,two,...,century,traffic,W,seeded,semi,rain,vendors,hour,stadium,death
0,0.405465,1.321756,2.70805,2.70805,inf,2.70805,2.70805,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,2.70805,inf,inf


In [ ]:
# 3. Реализовать метод, позволяющий векторизовать произвольный текст с использованием нейронных сетей (предлагается использовать стандартную реализацию модели w2v или glove). 
# Выбранную модель необходимо обучить на обучающей выборке.

In [ ]:
# 4. С использованием библиотечной реализации метода подсчета косинусного расстояния между векторными представлениями текста, 
# продемонстрировать на примерах, что для семантически близких слов модель генерирует вектора, для которых косинусное расстояние меньше, чем для семантически далеких токенов

In [ ]:
# 5. Применить какой-либо метод сокращения размерностей полученных одним из базовых способов векторизации, выбранным ранее (см. пункт 2), векторов

In [ ]:
# 6. С использованием разработанного метода подсчета косинусного расстояния сравнить эффективность метода векторизации с использованием 
#  нейронных сетей и эффективность базовых методов векторизации с последующим сокращением размерности.
# Сформулировать вывод о том, применение какого способа позволяет получить лучшие результаты на выбранном датасете.

In [ ]:
# 7. Реализовать метод, осуществляющий векторизацию произвольного текста по следующему алгоритму: https://github.com/Ko4eBHuK/nlp-23-spring/tree/main/tasks/task-03

In [ ]:
# 8. Выполнить векторизацию тестовой выборки с использованием метода, реализованного на предыдущем шаге.